In [ ]:
# default_exp grandmaster

# Git Grandmastery 
> Summary

By now, you should be able to navigate the *git help* pages and understand
almost everything. However, pinpointing the exact command required to solve a
given problem can be tedious. Perhaps I can save you some time: below are some
recipes I have needed in the past.

### Source Releases 

For my projects, Git tracks exactly the files I'd like to archive and release
to users. To create a tarball of the source code, I run:

In [ ]:
 ! git archive --format=tar --prefix=proj-1.2.3/ HEAD



### Commit What Changed 

Telling Git when you've added, deleted and renamed files is troublesome for
certain projects. Instead, you can type:



In [ ]:
 ! git add .
 ! git add -u



Git will look at the files in the current directory and work out the details by
itself. Instead of the second add command, run `git commit -a` if you also
intend to commit at this time. See *git help ignore* for how to specify files
that should be ignored.

You can perform the above in a single pass with:



In [ ]:
 ! git ls-files -d -m -o -z | xargs -0 git update-index --add --remove



The *-z* and *-0* options prevent ill side-effects from filenames containing
strange characters. As this command adds ignored files, you may want to use the
`-x` or `-X` option.

### My Commit Is Too Big! 

Have you neglected to commit for too long? Been coding furiously and forgotten
about source control until now? Made a series of unrelated changes, because
that's your style?

No worries. Run:



In [ ]:
 ! git add -p



For each edit you made, Git will show you the hunk of code that was changed,
and ask if it should be part of the next commit. Answer with "y" or "n". You
have other options, such as postponing the decision; type "?" to learn more.

Once you're satisfied, type



In [ ]:
 ! git commit



to commit precisely the changes you selected (the 'staged' changes). Make sure
you omit the *-a* option, otherwise Git will commit all the edits.

What if you've edited many files in many places? Reviewing each change one by
one becomes frustratingly mind-numbing. In this case, use *git add -i*, whose
interface is less straightforward, but more flexible. With a few keystrokes,
you can stage or unstage several files at a time, or review and select changes
in particular files only. Alternatively, run *git commit \--interactive* which
automatically commits after you're done.

### The Index: Git's Staging Area 

So far we have avoided Git's famous 'index', but we must now confront it to
explain the above. The index is a temporary staging area. Git seldom shuttles
data directly between your project and its history. Rather, Git first writes
data to the index, and then copies the data in the index to its final
destination.

For example, *commit -a* is really a two-step process. The first step places a
snapshot of the current state of every tracked file into the index. The second
step permanently records the snapshot now in the index. Committing without the
*-a* option only performs the second step, and only makes sense after running
commands that somehow change the index, such as *git add*.

Usually we can ignore the index and pretend we are reading straight from and writing straight to the history. On this occasion, we want finer control, so we manipulate the index. We place a snapshot of some, but not all, of our changes into the index, and then permanently record this carefully rigged snapshot.

### Don't Lose Your HEAD 

The HEAD tag is like a cursor that normally points at the latest commit, advancing with each new commit. Some Git commands let you move it. For example:



In [ ]:
 ! git reset HEAD~3



will move the HEAD three commits back. Thus all Git commands now act as if you hadn't made those last three commits, while your files remain in the present. See the help page for some applications.

But how can you go back to the future? The past commits know nothing of the future.

If you have the SHA1 of the original HEAD then:



In [ ]:
 ! git reset 1b6d



But suppose you never took it down? Don't worry: for commands like these, Git saves the original HEAD as a tag called ORIG_HEAD, and you can return safe and sound with:



In [ ]:
 ! git reset ORIG_HEAD



### HEAD-hunting 

Perhaps ORIG_HEAD isn't enough. Perhaps you've just realized you made a monumental mistake and you need to go back to an ancient commit in a long-forgotten branch.

By default, Git keeps a commit for at least two weeks, even if you ordered
Git to destroy the branch containing it. The trouble is finding the appropriate
hash. You could look at all the hash values in `.git/objects` and use trial
and error to find the one you want. But there's a much easier way.

Git records every hash of a commit it computes in `.git/logs`. The subdirectory `refs` contains the history of all activity on all branches, while the file `HEAD` shows every hash value it has ever taken. The latter can be used to find hashes of commits on branches that have been accidentally lopped off.

The reflog command provides a friendly interface to these log files. Try



In [ ]:
  ! git reflog



Instead of cutting and pasting hashes from the reflog, try:



In [ ]:
 ! git checkout "@{10 minutes ago}"



Or checkout the 5th-last visited commit via:



In [ ]:
 ! git checkout "@{5}"



See the ``Specifying Revisions'' section of *git help rev-parse* for more.

You may wish to configure a longer grace period for doomed commits. For
example:



In [ ]:
  ! git config gc.pruneexpire "30 days"



means a deleted commit will only be permanently lost once 30 days have passed
and *git gc* is run.

You may also wish to disable automatic invocations of *git gc*:



In [ ]:
  ! git config gc.auto 0



in which case commits will only be deleted when you run *git gc* manually.

### Building On Git 

In true UNIX fashion, Git's design allows it to be easily used as a low-level component of other programs, such as GUI and web interfaces, alternative command-line interfaces, patch managements tools, importing and conversion tools and so on. In fact, some Git commands are themselves scripts standing on the shoulders of giants. With a little tinkering, you can customize Git to suit your preferences.

One easy trick is to use built-in Git aliases to shorten your most frequently
used commands:



In [ ]:
  ! git config --global alias.co checkout
  ! git config --global --get-regexp alias  # display current aliases


  alias.co checkout


In [ ]:
  ! git co foo                              # same as 'git checkout foo'



Another is to print the current branch in the prompt, or window title.
Invoking



In [ ]:
  ! git symbolic-ref HEAD



shows the current branch name. In practice, you most likely want to remove
the "refs/heads/" and ignore errors:



In [ ]:
  ! git symbolic-ref HEAD 2> /dev/null | cut -b 12-



The +contrib+ subdirectory is a treasure trove of tools built on Git.
In time, some of them may be promoted to official commands. On Debian and
Ubuntu, this directory lives at +/usr/share/doc/git-core/contrib+.

One popular resident is +workdir/git-new-workdir+. Via clever symlinking, this script creates a new working directory whose history is shared with the original repository:



In [ ]:
  ! git-new-workdir an/existing/repo new/directory



The new directory and the files within can be thought of as a clone, except since the history is shared, the two trees automatically stay in sync. There's no need to merge, push, or pull.

### Daring Stunts 

These days, Git makes it difficult for the user to accidentally destroy data.
But if you know what you are doing, you can override safeguards for common
commands.

*Checkout*: Uncommitted changes cause checkout to fail. To destroy your changes, and checkout a given commit anyway, use the force flag:



In [ ]:
  ! git checkout -f HEAD^



On the other hand, if you specify particular paths for checkout, then there are no safety checks. The supplied paths are quietly overwritten. Take care if you use checkout in this manner.

*Reset*: Reset also fails in the presence of uncommitted changes. To force it through, run:



In [ ]:
  ! git reset --hard 1b6d



*Branch*: Deleting branches fails if this causes changes to be lost. To force a deletion, type:



In [ ]:
  ! git branch -D dead_branch  # instead of -d



Similarly, attempting to overwrite a branch via a move fails if data loss would ensue. To force a branch move, type:



In [ ]:
  ! git branch -M source target  # instead of -m



Unlike checkout and reset, these two commands defer data destruction. The
changes are still stored in the .git subdirectory, and can be retrieved by
recovering the appropriate hash from `.git/logs` (see "HEAD-hunting" above).
By default, they will be kept for at least two weeks.

*Clean*: Some git commands refuse to proceed because they're worried about
clobbering untracked files. If you're certain that all untracked files and
directories are expendable, then delete them mercilessly with:



In [ ]:
  ! git clean -f -d



Next time, that pesky command will work!

### Preventing Bad Commits 

Stupid mistakes pollute my repositories. Most frightening are missing files due
to a forgotten *git add*. Lesser transgressions are trailing whitespace and
unresolved merge conflicts: though harmless, I wish these never appeared on the
public record.

If only I had bought idiot insurance by using a _hook_ to alert me about these problems:



In [ ]:
 ! cd .git/hooks
 ! cp pre-commit.sample pre-commit  # Older Git versions: chmod +x pre-commit



Now Git aborts a commit if useless whitespace or unresolved merge conflicts are
detected.

For this guide, I eventually added the following to the beginning of the
*pre-commit* hook to guard against absent-mindedness:

 if git ls-files -o | grep '\.txt$'; then
   echo FAIL! Untracked .txt files.
   exit 1
 fi

Several git operations support hooks; see *git help hooks*. We activated the
sample *post-update* hook earlier when discussing Git over HTTP. This runs
whenever the head moves. The sample post-update script updates files Git needs
for communication over Git-agnostic transports such as HTTP.
